In [55]:
import pandas as pd
import numpy as np
import re

In [56]:
df_soccer = pd.read_csv('../data/soccer_reddit_unclean.csv')
df_fifa = pd.read_csv('../data/FIFA_reddit_unclean.csv')

In [57]:
df = pd.concat([df_soccer, df_fifa])
df

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,removed_by_category,crosspost_parent,crosspost_parent_list,is_gallery,distinguished,gallery_data,media_metadata,author_cakeday,banned_by,collections
0,[],False,AutoModerator,NaN,[],NaN,text,t2_6l4z3,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Chapulana,NaN,"[{'a': ':FC_Barcelona:', 'e': 'emoji', 'u': 'h...",:FC_Barcelona:,richtext,t2_g6pn4,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,joaorsilva,NaN,"[{'a': ':Benfica:', 'e': 'emoji', 'u': 'https:...",:Benfica:,richtext,t2_i4m77,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,klarstartpirat,NaN,"[{'a': ':CSC:', 'e': 'emoji', 'u': 'https://em...",:CSC:,richtext,t2_4cqs75de,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,ImamZain,NaN,"[{'a': ':r_soccer_user:', 'e': 'emoji', 'u': '...",:r_soccer_user:,richtext,t2_77p8qsj5,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,[],False,AdamMitchellWheyyyy,NaN,[],NaN,text,t2_1dqoucvc,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3996,[],False,Access_Kind,NaN,[],NaN,text,t2_8ra9vqm3,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3997,[],False,YakCarlo,NaN,[],NaN,text,t2_k90lsxnk,False,False,...,reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3998,[],False,Access_Kind,NaN,[],NaN,text,t2_8ra9vqm3,False,False,...,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df = df.reset_index(drop=True)

In [59]:
print(df[['author','subreddit', 'selftext', 'title']].head())
print(df[['author','subreddit', 'selftext', 'title']].tail())

           author subreddit  \
0   AutoModerator    soccer   
1       Chapulana    soccer   
2      joaorsilva    soccer   
3  klarstartpirat    soccer   
4        ImamZain    soccer   

                                            selftext  \
0  A place to discuss everything except the Engli...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                               title  
0                            Non-PL Daily Discussion  
1  [About the latest ECA/UEFA idea] PSG want Cham...  
2  [Record] Cristiano Ronaldo says he will be at ...  
3  Jack Grealish says English players get unfair ...  
4  [Pearce] The battle for Jude Bellingham… Dortm...  
                   author subreddit  \
7973  AdamMitchellWheyyyy      FIFA   
7974          Access_Kind      FIFA   
7975             YakCarlo    

## Remove all extra columns
Removing all columns other than Author, Subreddit, Post Title and Description.

In [60]:
df = df[['author','subreddit', 'selftext', 'title']]

If a post doesn't have any selftext, I am replacing the null value with an empty string.

In [61]:
df.isnull().sum()

author          0
subreddit       0
selftext     4890
title           0
dtype: int64

In [62]:
df.fillna('', inplace = True)

In [63]:
df.isnull().sum()

author       0
subreddit    0
selftext     0
title        0
dtype: int64

In [64]:
df['selftext'] = df['selftext'].replace({'[removed]':''})
df

,author,subreddit,selftext,title
0,AutoModerator,soccer,A place to discuss everything except the Engli...,Non-PL Daily Discussion
1,Chapulana,soccer,,[About the latest ECA/UEFA idea] PSG want Cham...
2,joaorsilva,soccer,,[Record] Cristiano Ronaldo says he will be at ...
3,klarstartpirat,soccer,,Jack Grealish says English players get unfair ...
4,ImamZain,soccer,,[Pearce] The battle for Jude Bellingham… Dortm...
...,...,...,...,...
7973,AdamMitchellWheyyyy,FIFA,"Hey all\n\nAs per the title, my connection is ...",Severe Connection Issues
7974,Access_Kind,FIFA,I am seeking advice on possible players to get...,Why are we not allowed to post out FUT screens...
7975,YakCarlo,FIFA,,FUT 19 TOTY Kit for anyone who wants a classy ...
7976,Access_Kind,FIFA,,Need advice on my Ligue 1 FUT


### Cleaning DataFrame
Removing all of the html breaks, and making all of our posts and titles lowercase

In [65]:
pattern = '(\\n)+[\* ]'
def remove_html(post):
    post = post.lower()
    return re.sub(pattern, ' ', post)

In [66]:
df['cleantext'] = df['selftext'].apply(remove_html)
df['cleantitle'] = df['title'].apply(remove_html)

In [67]:
df

,author,subreddit,selftext,title,cleantext,cleantitle
0,AutoModerator,soccer,A place to discuss everything except the Engli...,Non-PL Daily Discussion,a place to discuss everything except the engli...,non-pl daily discussion
1,Chapulana,soccer,,[About the latest ECA/UEFA idea] PSG want Cham...,,[about the latest eca/uefa idea] psg want cham...
2,joaorsilva,soccer,,[Record] Cristiano Ronaldo says he will be at ...,,[record] cristiano ronaldo says he will be at ...
3,klarstartpirat,soccer,,Jack Grealish says English players get unfair ...,,jack grealish says english players get unfair ...
4,ImamZain,soccer,,[Pearce] The battle for Jude Bellingham… Dortm...,,[pearce] the battle for jude bellingham… dortm...
...,...,...,...,...,...,...
7973,AdamMitchellWheyyyy,FIFA,"Hey all\n\nAs per the title, my connection is ...",Severe Connection Issues,"hey all\n\nas per the title, my connection is ...",severe connection issues
7974,Access_Kind,FIFA,I am seeking advice on possible players to get...,Why are we not allowed to post out FUT screens...,i am seeking advice on possible players to get...,why are we not allowed to post out fut screens...
7975,YakCarlo,FIFA,,FUT 19 TOTY Kit for anyone who wants a classy ...,,fut 19 toty kit for anyone who wants a classy ...
7976,Access_Kind,FIFA,,Need advice on my Ligue 1 FUT,,need advice on my ligue 1 fut


In [68]:
df.drop(columns = ['selftext', 'title'], inplace = True)

In [69]:
df.to_csv('../data/combined_reddit.csv', index = False)